In [30]:
import os
from dotenv import load_dotenv
from serpapi import GoogleSearch
from pydantic import Field
from typing import Optional

In [31]:
load_dotenv()
os.environ["SERPAPI_KEY"] = os.getenv("SERPAPI_KEY")

In [62]:
def get_return_flights(
    departure_token: str = Field(
        ..., description = "The token necessary to fetch return flight data"
    ),
    arrival_id: str = Field(
        ..., description = "The token necessary to fetch return flight data"
    ),
    departure_id: str = Field(
        ..., description = "The token necessary to fetch return flight data"
    ),
    outbound_date: str = Field(
        ..., description = "The token necessary to fetch return flight data"
    )
) -> list:

    params = {
        "engine": "google_flights",
        #"departure_token": departure_token,
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_date,
        "currency": "GBP",
        "type": 2,
        "api_key": os.environ["SERPAPI_KEY"]
    }

    search = GoogleSearch(params)
    results = search.get_dict()["best_flights"][:3]

    info = []
    for flight in results:
        return_info = {}
        return_info["departure_info"] = flight["flights"][0]["departure_airport"]
        return_info["arrival_info"] = flight["flights"][0]["arrival_airport"]
        return_info["price"] = flight["price"]
        info.append(return_info)
        
    return info

In [63]:
def get_flights(
    departure_id: str = Field(
        ..., description="The IATA code e.g. LGW for London Gatwick Airport"
    ), 
    arrival_id: str = Field(
        ..., description="The IATA code e.g. LGW for London Gatwick Airport"
    ), 
    outbound_date: str = Field(
        ..., description="The departure date in YYYY-MM-DD format"
    ), 
    return_date: Optional[str] = Field(
        None, description="The return date in YYYY-MM-DD format"
    ),
) -> dict:

        
    params = {
        "engine": "google_flights",
        "departure_id": departure_id,
        "arrival_id": arrival_id,
        "outbound_date": outbound_date,
        "return_date": return_date,
        "currency": "GBP",
        "hl": "en",
        "api_key": os.environ["SERPAPI_KEY"]
    }

    if return_date == None:
        params["type"] = 2
    
    search = GoogleSearch(params)
    results = search.get_dict()["best_flights"][:3]

    info = {"outbound_flights":[], "return_flights":[]}
    for flight in results:
        outbound_info = {}
        outbound_info["departure_info"] = flight["flights"][0]["departure_airport"]
        outbound_info["arrival_info"] = flight["flights"][0]["arrival_airport"]
        outbound_info["price"] = flight["price"]
        info["outbound_flights"].append(outbound_info)

    if return_date != None:
        departure_token = results[0]["departure_token"]
        return_info = get_return_flights(departure_token,departure_id,arrival_id,return_date)
        info["return_flights"].extend(return_info)


    return info

In [64]:
params = {
  "engine": "google_flights",
  "departure_id": "IST",
  "arrival_id": "LGW",
  "outbound_date": "2025-01-30",
  "return_date": "2025-02-03",
  "currency": "GB",
  "hl": "en",
  "api_key": "4fc75a76a7ec8a99deab65ec5847fd6850ebe277322015ea01d1d224b2ddbea4"
}

results = get_flights("IST","LGW","2025-02-10","2025-02-15")

In [65]:
results

{'outbound_flights': [{'departure_info': {'name': 'Istanbul Airport',
    'id': 'IST',
    'time': '2025-02-10 14:20'},
   'arrival_info': {'name': 'London Gatwick Airport',
    'id': 'LGW',
    'time': '2025-02-10 15:40'},
   'price': 214},
  {'departure_info': {'name': 'Istanbul Airport',
    'id': 'IST',
    'time': '2025-02-10 20:50'},
   'arrival_info': {'name': 'London Gatwick Airport',
    'id': 'LGW',
    'time': '2025-02-10 22:10'},
   'price': 219},
  {'departure_info': {'name': 'Istanbul Airport',
    'id': 'IST',
    'time': '2025-02-10 08:10'},
   'arrival_info': {'name': 'London Gatwick Airport',
    'id': 'LGW',
    'time': '2025-02-10 09:25'},
   'price': 329}],
 'return_flights': [{'departure_info': {'name': 'London Gatwick Airport',
    'id': 'LGW',
    'time': '2025-02-15 13:35'},
   'arrival_info': {'name': 'Istanbul Airport',
    'id': 'IST',
    'time': '2025-02-15 20:35'},
   'price': 176},
  {'departure_info': {'name': 'London Gatwick Airport',
    'id': 'LGW',
